In [1]:
"""
INDIVIDUAL ML MODEL CODE FOR EACH FUNCTION
Clean, runnable code showing exactly how each Week 7 input was generated
"""

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern
from scipy.optimize import minimize
from scipy.stats import norm

print("="*100)
print("INDIVIDUAL ML MODEL CODE - ALL 8 FUNCTIONS")
print("="*100)
print()

# ============================================================================
# F1 - LOCAL EXPLORATION (No Complex Model)
# ============================================================================

print("="*100)
print("F1 (2D) - LOCAL EXPLORATION AROUND BREAKTHROUGH")
print("="*100)
print()

code_f1 = '''
# F1: Week 6 found first non-zero at [0.45, 0.45] → 0.0128
# Strategy: Explore nearby region

import numpy as np

# Training data (6 weeks)
X_f1 = np.array([
    [0.10, 0.10],  # Week 1: 0.0
    [0.12, 0.08],  # Week 2: 0.0
    [0.21, 0.11],  # Week 3: 0.0
    [0.14, 0.14],  # Week 4: 0.0
    [0.08, 0.08],  # Week 5: 0.0
    [0.45, 0.45]   # Week 6: 0.0128 ← BREAKTHROUGH!
])
y_f1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0128])

# Week 6 found signal at [0.45, 0.45]
# Week 7: Explore nearby in positive gradient direction
breakthrough_point = np.array([0.45, 0.45])

# Test nearby points (gradient ascent)
candidates = [
    breakthrough_point + np.array([0.03, 0.03]),  # [0.48, 0.48]
    breakthrough_point + np.array([0.05, 0.00]),  # [0.50, 0.45]
    breakthrough_point + np.array([0.00, 0.05]),  # [0.45, 0.50]
]

# Choose: [0.48, 0.48] - symmetric increase
week7_f1 = np.array([0.48, 0.48])

print("Week 7 input:", week7_f1)
print("Strategy: Local gradient ascent from breakthrough")
'''

print(code_f1)
exec(code_f1)
print()

# ============================================================================
# F2 - GAUSSIAN PROCESS WITH EXPLOITATION
# ============================================================================

print("="*100)
print("F2 (2D) - GP WITH EXPLOITATION")
print("="*100)
print()

code_f2 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel

# Training data (6 weeks)
X_f2 = np.array([
    [0.10, 0.10],   # Week 1: 0.0892 (previous best)
    [0.12, 0.08],   # Week 2: 0.0705
    [0.21, 0.11],   # Week 3: 0.0295
    [0.14, 0.14],   # Week 4: 0.0150
    [0.08, 0.08],   # Week 5: 0.0463
    [0.111, 0.100]  # Week 6: 0.1300 ← NEW BEST!
])
y_f2 = np.array([0.0892, 0.0705, 0.0295, 0.0150, 0.0463, 0.1300])

# Known correlations (negative)
correlations_f2 = np.array([-0.530, -0.649])

# Build GP with ARD kernel (learns dimension importance)
# Initialize length scales from correlations
init_length_scales = 1.0 / (np.abs(correlations_f2) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f2 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f2.fit(X_f2, y_f2)

# Exploitation: Find maximum of GP mean
best_so_far = X_f2[-1]  # Week 6 best

# Generate candidates around best point
n_candidates = 100
candidates = []
for i in range(n_candidates):
    # Sample near best, with negative correlation guidance
    candidate = best_so_far.copy()
    candidate[0] += np.random.uniform(-0.02, 0.02)  # Small variation
    candidate[1] += np.random.uniform(-0.02, 0.02)
    candidate = np.clip(candidate, 0.0, 1.0)
    candidates.append(candidate)

candidates = np.array(candidates)

# Predict on all candidates
predictions = gp_f2.predict(candidates)

# Choose best predicted
best_idx = np.argmax(predictions)
week7_f2 = candidates[best_idx]

print("Week 7 input:", week7_f2)
print("Predicted output:", predictions[best_idx])
print("Learned length scales:", gp_f2.kernel_.k2.length_scale)
'''

print(code_f2)
exec(code_f2)
print()

# ============================================================================
# F3 - GP WITH BALANCED EXPLORATION
# ============================================================================

print("="*100)
print("F3 (3D) - GP WITH EXPECTED IMPROVEMENT")
print("="*100)
print()

code_f3 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f3 = np.array([
    [0.80, 0.80, 0.80],           # Week 1: -0.1055
    [0.95, 0.95, 0.95],           # Week 2: -0.0919
    [0.98, 0.99, 0.87],           # Week 3: -0.0856
    [0.948885, 0.965632, 0.808397],  # Week 4: -0.0786 (best)
    [1.01, 1.01, 0.82],           # Week 5: -1.1543 (penalty)
    [0.928, 0.832, 0.004]         # Week 6: -0.1161 (ML fail)
])
y_f3 = np.array([-0.1055, -0.0919, -0.0856, -0.0786, -1.1543, -0.1161])

# Known correlations
correlations_f3 = np.array([-0.14, -0.23, -0.35])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f3) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f3 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f3.fit(X_f3, y_f3)

# Expected Improvement acquisition function
def expected_improvement(X, gp, y_train, xi=0.01):
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    best_y = np.max(y_train)
    
    with np.errstate(divide='warn'):
        imp = mu - best_y - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    return ei

# Generate candidates near Week 4 best (avoid Week 6 mistake)
best_known = X_f3[3]  # Week 4
n_candidates = 200

candidates = []
for i in range(n_candidates):
    candidate = best_known.copy()
    candidate += np.random.uniform(-0.1, 0.1, size=3)
    candidate = np.clip(candidate, 0.0, 1.0)
    candidates.append(candidate)

candidates = np.array(candidates)

# Compute EI for all candidates
ei_scores = expected_improvement(candidates, gp_f3, y_f3)

# Choose highest EI
best_idx = np.argmax(ei_scores)
week7_f3 = candidates[best_idx]

print("Week 7 input:", week7_f3)
mu, std = gp_f3.predict([week7_f3], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Learned length scales:", gp_f3.kernel_.k2.length_scale)
'''

print(code_f3)
exec(code_f3)
print()

# ============================================================================
# F4 - GP WITH RECOVERY (CENTERED)
# ============================================================================

print("="*100)
print("F4 (4D) - GP FOR RECOVERY")
print("="*100)
print()

code_f4 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f4 = np.array([
    [0.5, 0.5, 0.5, 0.5],      # Week 1: -3.986 (BEST)
    [0.3, 0.3, 0.3, 0.3],      # Week 2: -4.306
    [0.44, 0.29, 0.35, 1.25],  # Week 3: -30.129 (disaster)
    [0.51, 0.60, 0.57, 0.01],  # Week 4: -12.492
    [0.66, 0.30, 0.30, 0.36],  # Week 5: -7.262
    [0.2, 0.2, 0.95, 0.4]      # Week 6: -19.009 (exploration fail)
])
y_f4 = np.array([-3.986, -4.306, -30.129, -12.492, -7.262, -19.009])

# Known correlations
correlations_f4 = np.array([-0.47, -0.37, -0.36, -0.77])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f4) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f4 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f4.fit(X_f4, y_f4)

# Expected Improvement for recovery
def expected_improvement(X, gp, y_train, xi=0.01):
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    best_y = np.max(y_train)
    
    with np.errstate(divide='warn'):
        imp = mu - best_y - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    return ei

# Generate candidates near center (Week 1 best)
center = np.array([0.5, 0.5, 0.5, 0.5])
n_candidates = 200

candidates = []
for i in range(n_candidates):
    # Stay close to center
    candidate = center + np.random.uniform(-0.1, 0.1, size=4)
    candidate = np.clip(candidate, 0.0, 1.0)
    candidates.append(candidate)

candidates = np.array(candidates)

# Compute EI
ei_scores = expected_improvement(candidates, gp_f4, y_f4)

# Choose highest EI
best_idx = np.argmax(ei_scores)
week7_f4 = candidates[best_idx]

print("Week 7 input:", week7_f4)
mu, std = gp_f4.predict([week7_f4], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Learned length scales:", gp_f4.kernel_.k2.length_scale)
print("Dimension importance: Dim 3 (54.6%), Dim 4 (37.7%)")
'''

print(code_f4)
exec(code_f4)
print()

# ============================================================================
# F5 - GP WITH LOCAL SEARCH (SHARP PEAK)
# ============================================================================

print("="*100)
print("F5 (4D) - GP WITH LOCAL SEARCH (CRITICAL!)")
print("="*100)
print()

code_f5 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.optimize import minimize

# Training data (6 weeks)
X_f5 = np.array([
    [0.3, 0.3, 0.3, 0.3],         # Week 1: 136.85
    [0.28, 0.32, 0.3, 0.29],      # Week 2: 137.29
    [0.344822, 0.264687, 0.374156, 0.203902],  # Week 3: 131.78
    [0.196828, 0.320017, 0.3, 0.289958],       # Week 4: 140.74
    [0.99, 0.9, 0.98, 0.93],      # Week 5: 5549.45 ← HUGE JUMP!
    [0.985, 0.905, 0.975, 0.925]  # Week 6: 5398.58 (slight regression)
])
y_f5 = np.array([136.85, 137.29, 131.78, 140.74, 5549.45, 5398.58])

# Known correlations (ALL very strong positive)
correlations_f5 = np.array([0.986, 0.997, 0.994, 0.992])

# Build GP - THIS IS CRITICAL!
# Week 5→6: tiny move (0.005) caused -151 point loss
# This indicates VERY sharp peak
init_length_scales = 1.0 / (np.abs(correlations_f5) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f5 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f5.fit(X_f5, y_f5)

# Local search from Week 5 best
week5_best = np.array([0.99, 0.9, 0.98, 0.93])

# Optimize GP mean prediction
def objective(x):
    return -gp_f5.predict([x])[0]

# Use L-BFGS-B for gradient-based optimization
result = minimize(
    objective,
    week5_best,
    bounds=[(0.0, 1.0)] * 4,
    method='L-BFGS-B'
)

week7_f5 = result.x

print("Week 7 input:", week7_f5)
mu, std = gp_f5.predict([week7_f5], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Improvement over Week 5 best:", mu[0] - 5549.45)
print()
print("Learned length scales:", gp_f5.kernel_.k2.length_scale)
print("Key insight: Dim 3 (96.6% importance)")
print()
print("Changes from Week 5 best:")
print("  Dim 1: 0.99 → {:.3f} ({:+.3f})".format(week7_f5[0], week7_f5[0] - 0.99))
print("  Dim 2: 0.90 → {:.3f} ({:+.3f})".format(week7_f5[1], week7_f5[1] - 0.90))
print("  Dim 3: 0.98 → {:.3f} ({:+.3f})".format(week7_f5[2], week7_f5[2] - 0.98))
print("  Dim 4: 0.93 → {:.3f} ({:+.3f})".format(week7_f5[3], week7_f5[3] - 0.93))
'''

print(code_f5)
exec(code_f5)
print()

# ============================================================================
# F6 - GP WITH BALANCED EI
# ============================================================================

print("="*100)
print("F6 (5D) - GP WITH EXPECTED IMPROVEMENT")
print("="*100)
print()

code_f6 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f6 = np.array([
    [0.75, 0.75, 0.75, 0.75, 0.75],  # Week 1: -1.521
    [0.3, 0.3, 0.3, 0.3, 0.3],       # Week 2: -1.139
    [0.49, 0.02, 0.45, 0.40, 0.32],  # Week 3: -1.123
    [0.69, 0.001, 0.04, 0.001, 0.001],  # Week 4: -2.067
    [0.26, 0.18, 0.50, 0.48, 0.41],  # Week 5: -1.092 (BEST)
    [0.1, 0.1, 0.7, 0.7, 0.6]        # Week 6: -1.231 (overshot)
])
y_f6 = np.array([-1.521, -1.139, -1.123, -2.067, -1.092, -1.231])

# Known correlations
correlations_f6 = np.array([-0.77, -0.54, 0.38, 0.42, 0.35])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f6) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f6 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f6.fit(X_f6, y_f6)

# Expected Improvement
def expected_improvement(X, gp, y_train, xi=0.01):
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    best_y = np.max(y_train)
    
    with np.errstate(divide='warn'):
        imp = mu - best_y - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    return ei

# Generate candidates near Week 5 best
week5_best = np.array([0.26, 0.18, 0.50, 0.48, 0.41])
n_candidates = 200

candidates = []
for i in range(n_candidates):
    # Use correlation guidance
    candidate = week5_best.copy()
    candidate[0] -= np.random.uniform(0, 0.1)  # Dim 1: negative corr
    candidate[1] -= np.random.uniform(0, 0.1)  # Dim 2: negative corr
    candidate[2] += np.random.uniform(0, 0.2)  # Dim 3: positive corr
    candidate[3] += np.random.uniform(0, 0.2)  # Dim 4: positive corr
    candidate[4] += np.random.uniform(0, 0.15) # Dim 5: positive corr
    candidate = np.clip(candidate, 0.0, 1.0)
    candidates.append(candidate)

candidates = np.array(candidates)

# Compute EI
ei_scores = expected_improvement(candidates, gp_f6, y_f6)

# Choose highest EI
best_idx = np.argmax(ei_scores)
week7_f6 = candidates[best_idx]

print("Week 7 input:", week7_f6)
mu, std = gp_f6.predict([week7_f6], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Learned length scales:", gp_f6.kernel_.k2.length_scale)
print("Key insight: Dim 5 (89.2% importance)")
'''

print(code_f6)
exec(code_f6)
print()

# ============================================================================
# F7 - GP WITH EXPLOITATION (PEER PATTERN)
# ============================================================================

print("="*100)
print("F7 (6D) - GP WITH EXPLOITATION (PEER INFORMED)")
print("="*100)
print()

code_f7 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel

# Training data (6 weeks)
X_f7 = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],    # Week 1: 0.000034
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],    # Week 2: 0.408
    [0.21, 0.19, 0.21, 0.19, 0.17, 0.19],  # Week 3: 0.347
    [0.08, 0.32, 0.15, 0.28, 0.41, 0.27],  # Week 4: 0.568
    [0.05, 0.50, 0.25, 0.20, 0.15, 0.85],  # Week 5: 0.836
    [0.06, 0.48, 0.25, 0.2, 0.4, 0.75]     # Week 6: 1.435 ← HUGE WIN!
])
y_f7 = np.array([0.000034, 0.408, 0.347, 0.568, 0.836, 1.435])

# Known correlations
correlations_f7 = np.array([-0.88, -0.50, -0.76, -0.77, -0.76, -0.15])

# Peer gradients (dimension importance from peer's analysis)
peer_gradients = np.array([0.12, 0.45, 0.08, 0.52, 0.23, 0.15])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f7) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f7 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f7.fit(X_f7, y_f7)

# Exploitation: maximize GP mean near Week 6 success
week6_best = np.array([0.06, 0.48, 0.25, 0.2, 0.4, 0.75])
n_candidates = 200

candidates = []
for i in range(n_candidates):
    # Small variations around Week 6
    candidate = week6_best.copy()
    candidate += np.random.uniform(-0.05, 0.05, size=6)
    candidate = np.clip(candidate, 0.0, 1.0)
    candidates.append(candidate)

candidates = np.array(candidates)

# Predict on all candidates
predictions = gp_f7.predict(candidates)

# Choose best predicted
best_idx = np.argmax(predictions)
week7_f7 = candidates[best_idx]

print("Week 7 input:", week7_f7)
mu, std = gp_f7.predict([week7_f7], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Improvement over Week 6:", mu[0] - 1.435)
print()
print("Learned length scales:", gp_f7.kernel_.k2.length_scale)
print("Key insight: Dim 6 (94% importance)")
print()
print("Comparison with peer's winning pattern:")
print("  Peer: [0.058, 0.492, 0.247, 0.218, 0.420, 0.731] → 1.365")
print("  Us (Week 6): [0.060, 0.480, 0.250, 0.200, 0.400, 0.750] → 1.435")
print("  Us (Week 7):", week7_f7)
'''

print(code_f7)
exec(code_f7)
print()

# ============================================================================
# F8 - GP WITH BALANCED EI
# ============================================================================

print("="*100)
print("F8 (8D) - GP WITH EXPECTED IMPROVEMENT")
print("="*100)
print()

code_f8 = '''
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f8 = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  # Week 1: 9.542
    [0.12, 0.09, 0.11, 0.1, 0.08, 0.13, 0.11, 0.09],  # Week 2: 9.554
    [0.29, 0.25, 0.02, 0.29, 0.14, 0.22, 0.25, 0.30],  # Week 3: 9.548
    [1.0, 0.001, 1.0, 0.001, 0.001, 1.0, 1.0, 0.001],  # Week 4: 4.180 (disaster)
    [0.05, 0.25, 0.25, 0.25, 0.25, 0.25, 0.05, 0.05],  # Week 5: 9.643
    [0.18, 0.15, 0.2, 0.15, 0.25, 0.15, 0.15, 0.18]    # Week 6: 9.676 (best)
])
y_f8 = np.array([9.542, 9.554, 9.548, 4.180, 9.643, 9.676])

# Known correlations
correlations_f8 = np.array([-0.98, 0.73, -0.98, 0.70, 0.72, -0.97, -0.98, 0.52])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f8) + 0.1)

kernel = ConstantKernel(1.0) * Matern(
    length_scale=init_length_scales,
    length_scale_bounds=(1e-2, 10.0),
    nu=2.5
)

gp_f8 = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-6,
    normalize_y=True
)
gp_f8.fit(X_f8, y_f8)

# Expected Improvement
def expected_improvement(X, gp, y_train, xi=0.01):
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    best_y = np.max(y_train)
    
    with np.errstate(divide='warn'):
        imp = mu - best_y - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    return ei

# Generate candidates near Week 6 best (avoid extremes like Week 4)
week6_best = np.array([0.18, 0.15, 0.2, 0.15, 0.25, 0.15, 0.15, 0.18])
n_candidates = 200

candidates = []
for i in range(n_candidates):
    # Small perturbations, prioritize Dim 3 and 5
    candidate = week6_best.copy()
    candidate += np.random.uniform(-0.05, 0.05, size=8)
    # Keep away from extremes (0 or 1)
    candidate = np.clip(candidate, 0.05, 0.95)
    candidates.append(candidate)

candidates = np.array(candidates)

# Compute EI
ei_scores = expected_improvement(candidates, gp_f8, y_f8)

# Choose highest EI
best_idx = np.argmax(ei_scores)
week7_f8 = candidates[best_idx]

print("Week 7 input:", week7_f8)
mu, std = gp_f8.predict([week7_f8], return_std=True)
print("Predicted output:", mu[0], "±", std[0])
print("Learned length scales:", gp_f8.kernel_.k2.length_scale)
print("Key insight: Dim 3 (71.4% importance)")
print("Peer insight: Dims 3, 5 most important")
'''

print(code_f8)
exec(code_f8)
print()

# ============================================================================
# Summary
# ============================================================================

print("="*100)
print("SUMMARY - ALL 8 MODELS")
print("="*100)
print()

summary = """
F1: Local exploration (no GP needed, just gradient ascent)
F2: GP + Exploitation (maximize mean prediction)
F3: GP + Expected Improvement (balanced exploration)
F4: GP + Expected Improvement (recovery near center)
F5: GP + Local Search (gradient descent on GP mean) ⭐ CRITICAL
F6: GP + Expected Improvement (correlation-guided)
F7: GP + Exploitation (build on Week 6 success)
F8: GP + Expected Improvement (avoid extremes)

ALL models use:
  • Matern kernel (nu=2.5) for flexibility
  • ARD (Automatic Relevance Determination) - learns dimension importance
  • Correlation initialization for length scales
  • 20 optimizer restarts for robustness
  • Output normalization for stability

Key Strategies:
  • Exploit when winning (F2, F7)
  • Explore when uncertain (F3, F8)
  • Recover when failing (F4, F6)
  • Local search for sharp peaks (F5)
"""

print(summary)

print("="*100)
print("ALL MODEL CODE COMPLETE!")
print("="*100)

INDIVIDUAL ML MODEL CODE - ALL 8 FUNCTIONS

F1 (2D) - LOCAL EXPLORATION AROUND BREAKTHROUGH


# F1: Week 6 found first non-zero at [0.45, 0.45] → 0.0128
# Strategy: Explore nearby region

import numpy as np

# Training data (6 weeks)
X_f1 = np.array([
    [0.10, 0.10],  # Week 1: 0.0
    [0.12, 0.08],  # Week 2: 0.0
    [0.21, 0.11],  # Week 3: 0.0
    [0.14, 0.14],  # Week 4: 0.0
    [0.08, 0.08],  # Week 5: 0.0
    [0.45, 0.45]   # Week 6: 0.0128 ← BREAKTHROUGH!
])
y_f1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0128])

# Week 6 found signal at [0.45, 0.45]
# Week 7: Explore nearby in positive gradient direction
breakthrough_point = np.array([0.45, 0.45])

# Test nearby points (gradient ascent)
candidates = [
    breakthrough_point + np.array([0.03, 0.03]),  # [0.48, 0.48]
    breakthrough_point + np.array([0.05, 0.00]),  # [0.50, 0.45]
    breakthrough_point + np.array([0.00, 0.05]),  # [0.45, 0.50]
]

# Choose: [0.48, 0.48] - symmetric increase
week7_f1 = np.array([0.48, 0.48])

print(

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Week 7 input: [0.85081639 0.9979837  0.84245168]
Predicted output: -0.2615574073025218 ± 0.4288432038939278
Learned length scales: [ 0.02016223 10.         10.        ]

F4 (4D) - GP FOR RECOVERY


import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f4 = np.array([
    [0.5, 0.5, 0.5, 0.5],      # Week 1: -3.986 (BEST)
    [0.3, 0.3, 0.3, 0.3],      # Week 2: -4.306
    [0.44, 0.29, 0.35, 1.25],  # Week 3: -30.129 (disaster)
    [0.51, 0.60, 0.57, 0.01],  # Week 4: -12.492
    [0.66, 0.30, 0.30, 0.36],  # Week 5: -7.262
    [0.2, 0.2, 0.95, 0.4]      # Week 6: -19.009 (exploration fail)
])
y_f4 = np.array([-3.986, -4.306, -30.129, -12.492, -7.262, -19.009])

# Known correlations
correlations_f4 = np.array([-0.47, -0.37, -0.36, -0.77])

# Build GP
init_length_scales = 1.0 / (np.abs(correlations_f4) + 0.1)

kernel = ConstantKernel(1.0)

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Week 7 input: [0.41157815 0.45830684 0.49380738 0.40190182]
Predicted output: -3.8093973332416677 ± 3.416270626533292
Learned length scales: [ 1.68369264 10.          0.20421943  0.29546285]
Dimension importance: Dim 3 (54.6%), Dim 4 (37.7%)

F5 (4D) - GP WITH LOCAL SEARCH (CRITICAL!)


import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.optimize import minimize

# Training data (6 weeks)
X_f5 = np.array([
    [0.3, 0.3, 0.3, 0.3],         # Week 1: 136.85
    [0.28, 0.32, 0.3, 0.29],      # Week 2: 137.29
    [0.344822, 0.264687, 0.374156, 0.203902],  # Week 3: 131.78
    [0.196828, 0.320017, 0.3, 0.289958],       # Week 4: 140.74
    [0.99, 0.9, 0.98, 0.93],      # Week 5: 5549.45 ← HUGE JUMP!
    [0.985, 0.905, 0.975, 0.925]  # Week 6: 5398.58 (slight regression)
])
y_f5 = np.array([136.85, 137.29, 131.78, 140.74, 5549.45, 5398.58])

# Known correlations (ALL very strong posit

...

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: Converg

Week 7 input: [0.20245658 0.11563142 0.67536571 0.67009724 0.48254256]
Predicted output: -1.1154870647741975 ± 0.04271793974705279
Learned length scales: [10.         10.         10.         10.          0.30164874]
Key insight: Dim 5 (89.2% importance)

F7 (6D) - GP WITH EXPLOITATION (PEER INFORMED)


import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel

# Training data (6 weeks)
X_f7 = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],    # Week 1: 0.000034
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],    # Week 2: 0.408
    [0.21, 0.19, 0.21, 0.19, 0.17, 0.19],  # Week 3: 0.347
    [0.08, 0.32, 0.15, 0.28, 0.41, 0.27],  # Week 4: 0.568
    [0.05, 0.50, 0.25, 0.20, 0.15, 0.85],  # Week 5: 0.836
    [0.06, 0.48, 0.25, 0.2, 0.4, 0.75]     # Week 6: 1.435 ← HUGE WIN!
])
y_f7 = np.array([0.000034, 0.408, 0.347, 0.568, 0.836, 1.435])

# Known correlations
correlations_f7 = np.array([-0.88, -0.50, -0.76, -0.7

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: Converg

Week 7 input: [0.03375881 0.46354552 0.24328114 0.18587307 0.41966973 0.734308  ]
Predicted output: 1.449608010933372 ± 0.07021635072502484
Improvement over Week 6: 0.01460801093337194

Learned length scales: [10.         10.         10.         10.         10.          0.12661705]
Key insight: Dim 6 (94% importance)

Comparison with peer's winning pattern:
  Peer: [0.058, 0.492, 0.247, 0.218, 0.420, 0.731] → 1.365
  Us (Week 6): [0.060, 0.480, 0.250, 0.200, 0.400, 0.750] → 1.435
  Us (Week 7): [0.03375881 0.46354552 0.24328114 0.18587307 0.41966973 0.734308  ]

F8 (8D) - GP WITH EXPECTED IMPROVEMENT


import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from scipy.stats import norm

# Training data (6 weeks)
X_f8 = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  # Week 1: 9.542
    [0.12, 0.09, 0.11, 0.1, 0.08, 0.13, 0.11, 0.09],  # Week 2: 9.554
    [0.29, 0.25, 0.02, 0.29, 0.14,

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: Converg